In [27]:
import numpy as np
import pandas as pd
import re, nltk, spacy, gensim

# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

/Users/sebastianwoitscheck/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [28]:
import json

df = pd.read_csv("data/anthology_conferences.csv", sep="|", keep_default_na=False)

#Add full text to dataframe, leave out proceedings title

#df = df.loc[0:500,:] #crashes when using the whole df - that's a lie i just made a mistake
#df = df.sample(frac=0.10)
#df.reset_index(drop=True, inplace=True)


notfound = []
df["fulltext"] = ""
missing = 0

for i, row in df.iterrows():
    fulltext = ""
    #df.at[i, "fulltext"] = []
    file = "json/" + row["pdf"] + ".json"
    try:
        with open(file) as jf:
            paper = json.load(jf)
            for item in paper["metadata"]["sections"]:
                if item["heading"] != None:
                    fulltext = fulltext + item["text"]   
    except:
        notfound.append(file) #catch missing jsons
        missing += 1
        fulltext = "nan"
    
    df.at[i, "fulltext"] = fulltext

print("missing {} rows".format(missing))  
display(df)
print(df.shape)
#print(notfound)

/Users/sebastianwoitscheck/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


missing 1523 rows


,url,publisher,address,year,month,editor,title,ENTRYTYPE,ID,pages,...,doi,isbn,language,number,note,pdf,abstract,semantic_scholar,semantic_scholar_authorIds,fulltext
0,https://www.aclweb.org/anthology/2020.acl-main.1,Association for Computational Linguistics,Online,2020,July,,Learning to Understand Child-directed and Adul...,inproceedings,gelderloos-etal-2020-learning,1--6,...,10.18653/v1/2020.acl-main.1,,,,,2020.acl-main.1.pdf,Speech directed to children differs from adult...,2020.acl-main.1.json,"['7805500', '2756960', '103538973']",Speech directed to children (CDS) differs from...
1,https://www.aclweb.org/anthology/2020.acl-main.2,Association for Computational Linguistics,Online,2020,July,,Predicting Depression in Screening Interviews ...,inproceedings,rinaldi-etal-2020-predicting,7--18,...,10.18653/v1/2020.acl-main.2,,,,,2020.acl-main.2.pdf,Despite the pervasiveness of clinical depressi...,2020.acl-main.2.json,"['19320780', '2457504', '37202877']",Depression is a dangerous disease that effects...
2,https://www.aclweb.org/anthology/2020.acl-main.3,Association for Computational Linguistics,Online,2020,July,,Coach: A Coarse-to-Fine Approach for Cross-dom...,inproceedings,liu-etal-2020-coach,19--25,...,10.18653/v1/2020.acl-main.3,,,,,2020.acl-main.3.pdf,As an essential task in task-oriented dialog s...,2020.acl-main.3.json,"['152613855', '9162688', '145011005', '40539650']",Slot filling models identify task-related slot...
3,https://www.aclweb.org/anthology/2020.acl-main.4,Association for Computational Linguistics,Online,2020,July,,Designing Precise and Robust Dialogue Response...,inproceedings,zhao-etal-2020-designing,26--33,...,10.18653/v1/2020.acl-main.4,,,,,2020.acl-main.4.pdf,Automatic dialogue response evaluator has been...,2020.acl-main.4.json,"['46887780', '1939089', '1717105']",Evaluation of conversational systems has been ...
4,https://www.aclweb.org/anthology/2020.acl-main.5,Association for Computational Linguistics,Online,2020,July,,Dialogue State Tracking with Explicit Slot Con...,inproceedings,ouyang-etal-2020-dialogue,34--40,...,10.18653/v1/2020.acl-main.5,,,,,2020.acl-main.5.pdf,Recent proposed approaches have made promising...,2020.acl-main.5.json,"['2070558572', '1753948811', '40539275', '2651...",Task-oriented dialogue systems assist users to...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15708,https://www.aclweb.org/anthology/Q13-1031,,,2013,,,Data-Driven Metaphor Recognition and Explanation,article,li-etal-2013-data,379--390,...,10.1162/tacl_a_00235,,,,,Q13-1031.pdf,Recognizing metaphors and identifying the sour...,tacl_a_00235.json,"['1730603', '1796651', '1745000']",A metaphor is a way of communicating. It enabl...
15709,https://www.aclweb.org/anthology/Q13-1032,,,2013,,,Powergrading: a Clustering Approach to Amplify...,article,basu-etal-2013-powergrading,391--402,...,10.1162/tacl_a_00236,,,,,Q13-1032.pdf,We introduce a new approach to the machine-ass...,tacl_a_00236.json,"['8611534', '2052757405', '1909300']",Increasing access to quality education is a gl...
15710,https://www.aclweb.org/anthology/Q13-1033,,,2013,,,Training Deterministic Parsers with Non-Determ...,article,goldberg-nivre-2013-training,403--414,...,10.1162/tacl_a_00237,,,,,Q13-1033.pdf,Greedy transition-based parsers are very fast ...,tacl_a_00237.json,"['2089067', '1720988']",Greedy transition-based parsers are easy to im...
15711,https://www.aclweb.org/anthology/Q13-1034,,,2013,,,Joint Morphological and Syntactic Analysis for...,article,bohnet-etal-2013-joint,415--428,...,10.1162/tacl_a_00238,,,,,Q13-1034.pdf,Joint morphological and syntactic analysis has...,tacl_a_00238.json,"['1678747', '1720988', '145605664', '144844761...",Syntactic parsing of natural language has witn...


(15713, 24)


In [29]:
df = df[df.fulltext != "nan"]
df.reset_index(drop=True, inplace=True)
display(df)
print(df.shape)

/Users/sebastianwoitscheck/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,url,publisher,address,year,month,editor,title,ENTRYTYPE,ID,pages,...,doi,isbn,language,number,note,pdf,abstract,semantic_scholar,semantic_scholar_authorIds,fulltext
0,https://www.aclweb.org/anthology/2020.acl-main.1,Association for Computational Linguistics,Online,2020,July,,Learning to Understand Child-directed and Adul...,inproceedings,gelderloos-etal-2020-learning,1--6,...,10.18653/v1/2020.acl-main.1,,,,,2020.acl-main.1.pdf,Speech directed to children differs from adult...,2020.acl-main.1.json,"['7805500', '2756960', '103538973']",Speech directed to children (CDS) differs from...
1,https://www.aclweb.org/anthology/2020.acl-main.2,Association for Computational Linguistics,Online,2020,July,,Predicting Depression in Screening Interviews ...,inproceedings,rinaldi-etal-2020-predicting,7--18,...,10.18653/v1/2020.acl-main.2,,,,,2020.acl-main.2.pdf,Despite the pervasiveness of clinical depressi...,2020.acl-main.2.json,"['19320780', '2457504', '37202877']",Depression is a dangerous disease that effects...
2,https://www.aclweb.org/anthology/2020.acl-main.3,Association for Computational Linguistics,Online,2020,July,,Coach: A Coarse-to-Fine Approach for Cross-dom...,inproceedings,liu-etal-2020-coach,19--25,...,10.18653/v1/2020.acl-main.3,,,,,2020.acl-main.3.pdf,As an essential task in task-oriented dialog s...,2020.acl-main.3.json,"['152613855', '9162688', '145011005', '40539650']",Slot filling models identify task-related slot...
3,https://www.aclweb.org/anthology/2020.acl-main.4,Association for Computational Linguistics,Online,2020,July,,Designing Precise and Robust Dialogue Response...,inproceedings,zhao-etal-2020-designing,26--33,...,10.18653/v1/2020.acl-main.4,,,,,2020.acl-main.4.pdf,Automatic dialogue response evaluator has been...,2020.acl-main.4.json,"['46887780', '1939089', '1717105']",Evaluation of conversational systems has been ...
4,https://www.aclweb.org/anthology/2020.acl-main.5,Association for Computational Linguistics,Online,2020,July,,Dialogue State Tracking with Explicit Slot Con...,inproceedings,ouyang-etal-2020-dialogue,34--40,...,10.18653/v1/2020.acl-main.5,,,,,2020.acl-main.5.pdf,Recent proposed approaches have made promising...,2020.acl-main.5.json,"['2070558572', '1753948811', '40539275', '2651...",Task-oriented dialogue systems assist users to...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14185,https://www.aclweb.org/anthology/Q13-1031,,,2013,,,Data-Driven Metaphor Recognition and Explanation,article,li-etal-2013-data,379--390,...,10.1162/tacl_a_00235,,,,,Q13-1031.pdf,Recognizing metaphors and identifying the sour...,tacl_a_00235.json,"['1730603', '1796651', '1745000']",A metaphor is a way of communicating. It enabl...
14186,https://www.aclweb.org/anthology/Q13-1032,,,2013,,,Powergrading: a Clustering Approach to Amplify...,article,basu-etal-2013-powergrading,391--402,...,10.1162/tacl_a_00236,,,,,Q13-1032.pdf,We introduce a new approach to the machine-ass...,tacl_a_00236.json,"['8611534', '2052757405', '1909300']",Increasing access to quality education is a gl...
14187,https://www.aclweb.org/anthology/Q13-1033,,,2013,,,Training Deterministic Parsers with Non-Determ...,article,goldberg-nivre-2013-training,403--414,...,10.1162/tacl_a_00237,,,,,Q13-1033.pdf,Greedy transition-based parsers are very fast ...,tacl_a_00237.json,"['2089067', '1720988']",Greedy transition-based parsers are easy to im...
14188,https://www.aclweb.org/anthology/Q13-1034,,,2013,,,Joint Morphological and Syntactic Analysis for...,article,bohnet-etal-2013-joint,415--428,...,10.1162/tacl_a_00238,,,,,Q13-1034.pdf,Joint morphological and syntactic analysis has...,tacl_a_00238.json,"['1678747', '1720988', '145605664', '144844761...",Syntactic parsing of natural language has witn...


(14190, 24)


In [30]:
df = df.loc[:,["title","abstract","fulltext"]]
import re # Load the regular expression library

#df['paper_text_processed'] = ""

for column in df:
    #print(column)
    # Remove punctuation
    df[column] = df[column].map(lambda x: re.sub('[,\.!?]', '', x))

    # Convert the titles to lowercase
    df[column] = df[column].map(lambda x: x.lower())

# Print out the first rows of papers
df

/Users/sebastianwoitscheck/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<>:9: DeprecationWarning: invalid escape sequence \.
<>:9: DeprecationWarning: invalid escape sequence \.
<ipython-input-30-e02c5139f1eb>:9: DeprecationWarning: invalid escape sequence \.
  df[column] = df[column].map(lambda x: re.sub('[,\.!?]', '', x))


,title,abstract,fulltext
0,learning to understand child-directed and adul...,speech directed to children differs from adult...,speech directed to children (cds) differs from...
1,predicting depression in screening interviews ...,despite the pervasiveness of clinical depressi...,depression is a dangerous disease that effects...
2,coach: a coarse-to-fine approach for cross-dom...,as an essential task in task-oriented dialog s...,slot filling models identify task-related slot...
3,designing precise and robust dialogue response...,automatic dialogue response evaluator has been...,evaluation of conversational systems has been ...
4,dialogue state tracking with explicit slot con...,recent proposed approaches have made promising...,task-oriented dialogue systems assist users to...
...,...,...,...
14185,data-driven metaphor recognition and explanation,recognizing metaphors and identifying the sour...,a metaphor is a way of communicating it enable...
14186,powergrading: a clustering approach to amplify...,we introduce a new approach to the machine-ass...,increasing access to quality education is a gl...
14187,training deterministic parsers with non-determ...,greedy transition-based parsers are very fast ...,greedy transition-based parsers are easy to im...
14188,joint morphological and syntactic analysis for...,joint morphological and syntactic analysis has...,syntactic parsing of natural language has witn...


In [34]:
# Convert to list
data = df.abstract.values.tolist()

# Remove Emails
#data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

# Remove anything in ()
data = [re.sub(r'\(.*?\)', "", sent) for sent in data]

# Remove links
data = [re.sub(r"\S*https?:\S*", "", sent) for sent in data]

pprint(data[:1])

/Users/sebastianwoitscheck/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<>:8: DeprecationWarning: invalid escape sequence \s
<>:8: DeprecationWarning: invalid escape sequence \s
<ipython-input-34-dcc7fb2852c7>:8: DeprecationWarning: invalid escape sequence \s
  data = [re.sub('\s+', ' ', sent) for sent in data]


['speech directed to children differs from adultdirected speech in linguistic '
 'aspects such as repetition word choice and sentence length as well as in '
 'aspects of the speech signal itself such as prosodic and phonemic variation '
 'human language acquisition research indicates that child-directed speech '
 'helps language learners this study explores the effect of child-directed '
 'speech when learning to extract semantic information from speech directly we '
 'compare the task performance of models trained on adult-directed speech  and '
 'child-directed speech  we find indications that cds helps in the initial '
 'stages of learning but eventually models trained on ads reach comparable '
 'task performance and generalize better the results suggest that this is at '
 'least partially due to linguistic rather than acoustic properties of the two '
 'registers as we see the same pattern when looking at models trained on '
 'acoustically comparable synthetic speech']


In [35]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

/Users/sebastianwoitscheck/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[['speech', 'directed', 'to', 'children', 'differs', 'from', 'adultdirected', 'speech', 'in', 'linguistic', 'aspects', 'such', 'as', 'repetition', 'word', 'choice', 'and', 'sentence', 'length', 'as', 'well', 'as', 'in', 'aspects', 'of', 'the', 'speech', 'signal', 'itself', 'such', 'as', 'prosodic', 'and', 'phonemic', 'variation', 'human', 'language', 'acquisition', 'research', 'indicates', 'that', 'child', 'directed', 'speech', 'helps', 'language', 'learners', 'this', 'study', 'explores', 'the', 'effect', 'of', 'child', 'directed', 'speech', 'when', 'learning', 'to', 'extract', 'semantic', 'information', 'from', 'speech', 'directly', 'we', 'compare', 'the', 'task', 'performance', 'of', 'models', 'trained', 'on', 'adult', 'directed', 'speech', 'and', 'child', 'directed', 'speech', 'we', 'find', 'indications', 'that', 'cds', 'helps', 'in', 'the', 'initial', 'stages', 'of', 'learning', 'but', 'eventually', 'models', 'trained', 'on', 'ads', 'reach', 'comparable', 'task', 'performance', 'an

In [36]:
# ... Dauert 1000 Jahre

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# Run in terminal: python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:2])


/Users/sebastianwoitscheck/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['speech direct child differ adultdirecte speech linguistic aspect such repetition word choice sentence length as well aspect speech signal such prosodic phonemic human language acquisition research indicate child direct speech help language learner study explore effect child direct speech when learn extract semantic information speech directly compare task performance model train adult direct speech child direct speech find indication cd help initial stage learning eventually model train ad reach comparable task performance generalize well result suggest least partially linguistic rather acoustic property register see same pattern when look model train acoustically comparable synthetic speech', 'pervasiveness clinical depression modern society professional help remain highly stigmatize inaccessible expensive accurately diagnose depression difficult require time intensive interview assessment analysis hence automate method can assess linguistic pattern interview could help psychiatric 

In [49]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'et', 'al', 'text', 'feature', 'language', 'measure', 'result', 'score', 'word', 'set', 'work', 'model'])

vectorizer = CountVectorizer(analyzer='word',       
                             min_df=2,                        # minimum reqd occurences of a word 
                             stop_words=stop_words,  # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             # max_features=50000,             # max number of uniq words
                            )

data_vectorized = vectorizer.fit_transform(data_lemmatized)

/Users/sebastianwoitscheck/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [50]:
# Materialize the sparse data
data_dense = data_vectorized.todense()

# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")

/Users/sebastianwoitscheck/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Sparsicity:  0.5431760784906843 %


In [51]:
# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=50,               # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes
LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=20, n_jobs=-1, perp_tol=0.1,
             random_state=100, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

/Users/sebastianwoitscheck/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


LatentDirichletAllocation(learning_method='online', n_components=50, n_jobs=-1,
                          random_state=100)


LatentDirichletAllocation(learning_method='online', n_components=20, n_jobs=-1,
                          random_state=100)

In [52]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(data_vectorized))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(data_vectorized))

# See model parameters
pprint(lda_model.get_params())


/Users/sebastianwoitscheck/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Log Likelihood:  -7546714.298504831
Perplexity:  1945.898247196331
{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'online',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 10,
 'mean_change_tol': 0.001,
 'n_components': 50,
 'n_jobs': -1,
 'perp_tol': 0.1,
 'random_state': 100,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


In [41]:
# Beware, this takes ages... (5-infinity minutes depending on search params)

# Define Search Param
search_params = {'n_components': [25, 50], 'learning_decay': [.5, .9]}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
model.fit(data_vectorized)


/Users/sebastianwoitscheck/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/Users/sebastianwoitscheck/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:443: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  fold_sizes = n

GridSearchCV(estimator=LatentDirichletAllocation(),
             param_grid={'learning_decay': [0.5, 0.9],
                         'n_components': [25, 50]})

In [42]:
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

results = pd.DataFrame.from_dict(model.cv_results_)
results.head()

/Users/sebastianwoitscheck/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Best Model's Params:  {'learning_decay': 0.5, 'n_components': 25}
Best Log Likelihood Score:  -1449579.0641778666
Model Perplexity:  997.6050983500238


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_decay,param_n_components,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,52.028048,5.838635,1.065637,0.148706,0.5,25,"{'learning_decay': 0.5, 'n_components': 25}",-1.476925e+06,-1.489820e+06,-1.337970e+06,-1.434703e+06,-1.508478e+06,-1.449579e+06,60850.532286,1
1,58.128722,1.649686,1.211123,0.050504,0.5,50,"{'learning_decay': 0.5, 'n_components': 50}",-1.542311e+06,-1.555828e+06,-1.403410e+06,-1.500301e+06,-1.570333e+06,-1.514437e+06,60234.919237,4
2,54.657626,2.091341,1.072301,0.058363,0.9,25,"{'learning_decay': 0.9, 'n_components': 25}",-1.479311e+06,-1.494411e+06,-1.336574e+06,-1.438778e+06,-1.509365e+06,-1.451688e+06,62181.556582,2
3,58.877147,1.529986,1.167151,0.044776,0.9,50,"{'learning_decay': 0.9, 'n_components': 50}",-1.541405e+06,-1.554090e+06,-1.402191e+06,-1.501437e+06,-1.571689e+06,-1.514163e+06,60572.540661,3


In [43]:
# Create Document - Topic Matrix
lda_output = best_lda_model.transform(data_vectorized)

# column names
topicnames = ["Topic" + str(i) for i in range(best_lda_model.n_components)]

# index names
docnames = ["Doc" + str(i) for i in range(len(data))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics

/Users/sebastianwoitscheck/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,Topic10,Topic11,Topic12,Topic13,Topic14,Topic15,Topic16,Topic17,Topic18,Topic19,Topic20,Topic21,Topic22,Topic23,Topic24,dominant_topic
Doc0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.990000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,17
Doc1,0.000000,0.000000,0.000000,0.000000,0.060000,0.000000,0.000000,0.000000,0.310000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.110000,0.340000,0.000000,0.000000,0.090000,0.000000,0.090000,0.000000,0.000000,0.000000,16
Doc2,0.000000,0.150000,0.000000,0.280000,0.320000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.160000,4
Doc3,0.000000,0.000000,0.000000,0.200000,0.140000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.140000,0.000000,0.000000,0.000000,0.000000,0.000000,9
Doc4,0.000000,0.000000,0.000000,0.000000,0.530000,0.000000,0.000000,0.000000,0.120000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.240000,0.000000,0.000000,0.000000,0.060000,0.000000,0.030000,4
Doc5,0.000000,0.000000,0.000000,0.000000,0.190000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.220000,0.000000,0.000000,0.000000,0.000000,0.000000,0.410000,0.000000,0.000000,0.000000,0.030000,0.000000,0.130000,18
Doc6,0.000000,0.170000,0.000000,0.000000,0.490000,0.020000,0.000000,0.000000,0.000000,0.070000,0.000000,0.060000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.140000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4
Doc7,0.000000,0.000000,0.000000,0.000000,0.380000,0.000000,0.000000,0.150000,0.000000,0.170000,0.000000,0.040000,0.000000,0.030000,0.000000,0.000000,0.190000,0.020000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,4
Doc8,0.000000,0.220000,0.000000,0.000000,0.340000,0.000000,0.000000,0.000000,0.150000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,4
Doc9,0.000000,0.130000,0.000000,0.070000,0.370000,0.000000,0.000000,0.000000,0.000000,0.210000,0.000000,0.000000,0.150000,0.000000,0.000000,0.000000,0.000000,0.000000,0.070000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4


In [44]:
df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Documents")
df_topic_distribution.columns = ['Topic Num', 'Num Documents']
df_topic_distribution

/Users/sebastianwoitscheck/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Topic Num,Num Documents
0,3,1031
1,20,868
2,18,848
3,19,814
4,5,756
5,11,723
6,14,711
7,8,689
8,12,646
9,7,635


In [45]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(best_lda_model, data_vectorized, vectorizer, mds='tsne')
panel

/Users/sebastianwoitscheck/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/Users/sebastianwoitscheck/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/pairwise.py:58: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
/Users/sebastianwoitscheck/opt/anaconda3/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:349: DeprecationWarning: `

PreparedData(topic_coordinates=               x          y  topics  cluster      Freq
topic                                                 
3     -31.058315  29.500683       1        1  6.946425
18    -21.461353 -17.436399       2        1  6.006428
20     -6.096575 -39.141697       3        1  5.199685
21     56.071609  -9.400822       4        1  5.159987
14     61.351185 -56.833744       5        1  4.841923
11     17.821402  57.413609       6        1  4.750047
19     49.250862  50.355305       7        1  4.740005
2     -13.172927  53.618752       8        1  4.733626
8      31.702734 -64.520691       9        1  4.705228
5      14.361623   7.507676      10        1  4.624848
7      29.184048 -13.613249      11        1  4.338671
1      19.284979 -39.023891      12        1  4.229856
12      4.356016 -15.153996      13        1  4.172723
23    -12.708655   6.676149      14        1  3.614890
22    -46.884609 -28.260098      15        1  3.362110
16    -28.764826 -54.799591      16        1  3.296439
10     62.600399  25.238491      17        1  3.197267
4     -43.313934   3.224185      18        1  3.142456
24     44.929001 -34.219784      19        1  3.137210
6      81.611572   3.921941      20        1  3.039150
15      0.798218  29.447706      21        1  3.001632
9       2.051419 -67.215591      22        1  2.881046
17     27.693497  31.413803      23        1  2.820607
13     40.470249   9.942824      24        1  2.381976
0      76.766655 -28.811687      25        1  1.675764, topic_info=             Term         Freq        Total Category  logprob  loglift
3725  translation  5209.000000  5209.000000  Default  30.0000  30.0000
3231     semantic  4967.000000  4967.000000  Default  29.0000  29.0000
3017     relation  3373.000000  3373.000000  Default  28.0000  28.0000
1218       entity  2972.000000  2972.000000  Default  27.0000  27.0000
2899     question  3206.000000  3206.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
3981         well    79.723100  4340.658725  Topic25  -5.3179   0.0917
2795      problem    78.048239  4005.670408  Topic25  -5.3392   0.1508
2848      propose    74.591763  8787.448974  Topic25  -5.3845  -0.6801
2000        large    67.678288  3400.729936  Topic25  -5.4817   0.1719
1562         good    65.083545  1987.025100  Topic25  -5.5208   0.6702

[1692 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
0         8  0.152967  abbreviation
0        20  0.826019  abbreviation
4        21  0.968143          absa
10       12  0.004244   abstractive
10       13  0.993149   abstractive
...     ...       ...           ...
4007     21  0.490538       wordnet
4008      5  0.095622        worker
4008      9  0.669353        worker
4008     18  0.207181        worker
4020     21  0.989316           wsd

[8901 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 19, 21, 22, 15, 12, 20, 3, 9, 6, 8, 2, 13, 24, 23, 17, 11, 5, 25, 7, 16, 10, 18, 14, 1])

In [46]:
# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(best_lda_model.components_)

# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names()
df_topic_keywords.index = topicnames

# View
df_topic_keywords.head()


/Users/sebastianwoitscheck/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,abbreviation,ability,ablation,able,absa,absence,absent,absolute,abstract,abstraction,...,wrong,wrt,wsd,xml,year,yelp,yet,yield,young,zeroshot
Topic0,0.040000,0.040000,0.040000,8.022162,0.04,1.482143,0.040000,2.544996,0.040000,0.040000,...,0.040000,0.04,0.04,0.040000,0.040001,9.820753,0.040000,11.194010,0.04,0.04000
Topic1,0.040000,33.891304,13.384696,55.839302,0.04,1.556907,0.040000,2.747651,0.304525,0.040000,...,2.701349,0.04,0.04,0.040000,8.423684,0.040000,16.589004,33.001696,0.04,0.04000
Topic2,10.609473,1.259088,0.040000,20.453474,0.04,0.040000,0.040027,1.564765,25.087100,15.095622,...,13.417985,0.04,0.04,0.040000,4.476998,0.040000,11.453817,26.949585,0.04,0.04000
Topic3,0.040000,37.914362,2.213150,16.592151,0.04,14.151751,2.088361,6.448736,0.072543,0.040000,...,0.040000,0.04,0.04,0.040000,6.051516,0.040000,9.775180,74.521400,0.04,13.78697
Topic4,0.040000,17.474553,0.691027,48.232759,0.04,0.040001,0.040000,0.040000,0.040000,0.040000,...,1.193598,0.04,0.04,0.503206,4.707899,0.040000,3.377535,10.673024,0.04,0.04000


In [47]:
# Show top n keywords for each topic
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=15)        

# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

/Users/sebastianwoitscheck/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,resolution,coreference,approach,story,cluster,pronoun,base,context,narrative,ion,antecedent,system,show,gram,paper
Topic 1,train,training,neural,sequence,task,propose,pre,domain,sentence,datum,image,approach,encoder,generate,generation
Topic 2,rule,syntactic,structure,morphological,form,analysis,tree,system,sentence,lexical,segmentation,linguistic,level,base,paper
Topic 3,datum,label,training,method,learn,learning,domain,classification,task,approach,propose,train,supervise,classifier,show
Topic 4,dialogue,response,system,conversation,utterance,user,human,dialog,speech,conversational,propose,task,domain,spoken,agent
Topic 5,embedding,vector,representation,similarity,method,space,semantic,learn,task,propose,embed,base,show,approach,cluster
Topic 6,query,domain,system,web,question,search,retrieval,answer,method,information,rank,document,candidate,approach,paper
Topic 7,performance,show,task,datum,character,domain,improve,training,train,state,art,improvement,well,representation,parameter
Topic 8,question,task,answer,dataset,knowledge,comprehension,state,performance,new,read,reasoning,challenge,show,natural,propose
Topic 9,system,bias,evaluation,human,style,fact,gender,dataset,claim,metric,student,knowledge,show,make,generation


In [48]:
# Define function to predict topic for a given text document.
nlp = spacy.load('en', disable=['parser', 'ner'])

def predict_topic(text, nlp=nlp):
    global sent_to_words
    global lemmatization

    # Step 1: Clean with simple_preprocess
    mytext_2 = list(sent_to_words(text))

    # Step 2: Lemmatize
    mytext_3 = lemmatization(mytext_2, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

    # Step 3: Vectorize transform
    mytext_4 = vectorizer.transform(mytext_3)

    # Step 4: LDA Transform
    topic_probability_scores = best_lda_model.transform(mytext_4)
    topic = df_topic_keywords.iloc[np.argmax(topic_probability_scores), :].values.tolist()
    return topic, topic_probability_scores

# Predict the topic
mytext = ["bullshit canon"]
topic, prob_scores = predict_topic(text = mytext)
print(topic)

/Users/sebastianwoitscheck/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['resolution', 'coreference', 'approach', 'story', 'cluster', 'pronoun', 'base', 'context', 'narrative', 'ion', 'antecedent', 'system', 'show', 'gram', 'paper']
